# **Example Usage of Pangeo-Fish Software**


**Overview:**
This Jupyter notebook demonstrates the usage of the Pangeo-Fish software, a tool designed for analyzing biologging data in reference to Earth Observation (EO) data.

The biologging data consist of Data Storage Tag (DST), along with release and recapture time and location of the species in question. Both biologging data and the reference EO data are accessible with https and the access methods are incorporated in this notebook.   

**Purpose:**
By executing this notebook, users will learn how to set up a workflow for utilizing the Pangeo-Fish software. The workflow consists of 9 steps which are described below:

1. **Configure the Notebook:** Prepare the notebook environment for analysis.
2. **Compare Reference Model with DST Information:** Analyze and compare data from the reference model with information from the biologging data of the species in question. 
3. **Regrid the Grid from Reference Model Grid to Healpix Grid:** Transform the grid from the reference model to the Healpix grid for further analysis.
4. **Construct Emission Matrix:** Create an emission matrix based on the transformed grid.
5. **Replace emission for flagged tags:** If the tags are flagged for warm water, then it use the detection file associated and change the flagged timestamps.
6. **Combine and Normalize Emission Matrix:** Merge the emission matrix and normalize it for further processing.
7. **Estimate Model Parameters:** Determine the parameters of the model based on the normalized emission matrix.
8. **Compute State Probabilities and Tracks:** Calculate the probability distribution of the species in question and compute the tracks.
9. **Visualization:** Visualize the results of the analysis for interpretation and insight.

Throughout this notebook, users will gain practical experience in setting up and executing a workflow using Pangeo-Fish, enabling them to apply similar methodologies to their own biologging data analysis tasks.



## 1. **Configure the Notebook:** Prepare the notebook environment for analysis.

In this step, we sets up the notebook environment for analysis. It includes installing necessary packages, importing required libraries, setting up parameters, and configuring the cluster for distributed computing. It also retrieves the tag data needed for analysis.

    

In [4]:
# Import necessary libraries and modules.
import xarray as xr
from pint_xarray import unit_registry as ureg
from pangeo_fish.io import open_tag
import hvplot.xarray
import intake
import pandas as pd

/srv/conda/envs/notebook/lib/python3.11/site-packages/movingpandas/__init__.py:37: UserWarning: Missing optional dependencies. To use the trajectory smoother classes please install Stone Soup (see https://stonesoup.readthedocs.io/en/latest/#installation).
  warnings.warn(e.msg, UserWarning)


In [5]:
#
# Set up execution parameters for the analysis.
#
# Note: This cell is tagged as parameters, allowing automatic updates when configuring with papermil.

# tag_name corresponds to the name of the biologging tag name (DST identification number), 
# which is also a path for storing all the information for the specific fish tagged with tag_name.
# tag_name = "AD_A11849"
# tag_name = "SV_A11957"


tag_list = ['NO_A12710','CB_A11036','LT_A11385','SQ_A10684','AD_A11177','PB_A12063','NO_A12742','DK_A10642','CB_A11071']
tag_name = tag_list[8]
tag_name = "LT_A11338"

cloud_root = "s3://gfts-ifremer/tags/bargip"

# tag_root specifies the root URL for tag data used for this computation.
tag_root = f"{cloud_root}/cleaned"

# catalog_url specifies the URL for the catalog for reference data used.
catalog_url = "s3://gfts-ifremer/copernicus_catalogs/master.yml"

# scratch_root specifies the root directory for storing output files.
scratch_root = f"{cloud_root}/tracks"


# storage_options specifies options for the filesystem storing output files.
storage_options = {
    'anon': False, 
    # 'profile' : "gfts",
    'client_kwargs': {
        "endpoint_url": "https://s3.gra.perf.cloud.ovh.net",
        "region_name": "gra",
    }
}

# if you are using local file system, activate following two lines
folder_name = "../toto"
storage_options =None
scratch_root = f"/home/jovyan/notebooks/papermill/{folder_name}"

# Default chunk value for time dimension.  This values depends on the configuration of your dask cluster.
chunk_time=24

#
# Parameters for step 2. **Compare Reference Model with DST Information:**
#
# bbox, bounding box, defines the latitude and longitude range for the analysis area.
bbox = {"latitude": [40, 56], "longitude": [-13, 5]} 

# relative_depth_threshold defines the acceptable fish depth relative to the maximum tag depth.
# It determines whether the fish can be considered to be in a certain location based on depth.
relative_depth_threshold = 0.8

#
# Parameters for step 3. **Regrid the Grid from Reference Model Grid to Healpix Grid:** 
#
# Distance filepath is the path to the coastal distance file.
distance_filepath = "s3://gfts-ifremer/tags/distance2coast.zarr"

# distance_scale_factor scales the squared distance in the exponential decay function.
distance_scale_factor = 0.01

# nside defines the resolution of the healpix grid used for regridding.
nside = 4096 #*2

# rot defines the rotation angles for the healpix grid.
rot = {"lat": 0, "lon": 30}

# min_vertices sets the minimum number of vertices for a valid transcription for regridding.
min_vertices = 1

#
# Parameters for step 4. **Construct Emission Matrix:**
#
# differences_std sets the standard deviation for scipy.stats.norm.pdf.
# It expresses the estimated certainty of the field of difference.
differences_std = 0.75

# recapture_std sets the covariance for recapture event.
# It shows the certainty of the final recapture area if it is known.
recapture_std = 1e-2

# earth_radius defines the radius of the Earth used for distance calculations.
earth_radius = ureg.Quantity(6371, "km")

# maximum_speed sets the maximum allowable speed for the tagged fish.
maximum_speed = ureg.Quantity(60, "km / day")

# adjustment_factor adjusts parameters for a more fuzzy search.
# It will factor the allowed maximum displacement of the fish.
adjustment_factor = 5  

# truncate sets the truncating factor for computed maximum allowed sigma for convolution process.
truncate = 4

#
# Parameters for step 5. **Compute Additional Emission Probability Matrix:**
#


# buffer_size sets the size of the powerplant warm plume.
buffer_size = ureg.Quantity(1000, "m")
# powerplant_flag is a boolean that states if the fish has swam in warm plume


#
# Parameters for step 7. **Estimate Model Parameters:** 
#
# tolerance sets the tolerance level for optimised parameter serarch computation.
tolerance = 1e-3

#
# Parameters for step 8. **Compute State Probabilities and Tracks:** 
#
# track_modes defines the modes for track calculation.
track_modes = ["mean", "mode"]

# additional_track_quantities sets quantities to compute for tracks using moving pandas.
additional_track_quantities = ["speed", "distance"]


#
# Parameters for step 9. **Visualization:** 
#
# time_step defines for each time_step value we visualize state and emission matrix. 
time_step = 3


# Define target root directories for storing analysis results.
target_root = f"{scratch_root}/{tag_name}"

# Defines default chunk size for optimisation.  
default_chunk = {"time": chunk_time, "lat": -1, "lon": -1}
default_chunk_xy = {"time": chunk_time, "x": -1, "y": -1}


In [6]:
# Define target root directories for storing analysis results.
target_root = f"{scratch_root}/{tag_name}"

In [7]:
target_root

'/home/jovyan/notebooks/papermill/../toto/LT_A11338'

In [8]:
tag_root

's3://gfts-ifremer/tags/bargip/cleaned'

In [9]:
warm_plume = pd.read_csv("s3://gfts-ifremer/tags/bargip/bar_flag_warm_plume.txt",sep = "\t")
warm_list = list(warm_plume[warm_plume["warm_plume"]==True]["tag_name"])

if tag_name in  warm_list : 
    powerplant_flag = True
else : 
    powerplant_flag = False

In [10]:
if powerplant_flag:
    detection_file = f"{tag_root}/{tag_name}/detection.csv"
    powerplant_file = f"{cloud_root}/nuclear_plant_loc.csv"

In [11]:
# Set up a local cluster for distributed computing.
from distributed import LocalCluster
cluster = LocalCluster()
client = cluster.get_client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 3
Total threads: 6,Total memory: 24.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:45741,Workers: 3
Dashboard: http://127.0.0.1:8787/status,Total threads: 6
Started: Just now,Total memory: 24.00 GiB
Comm: tcp://127.0.0.1:33491,Total threads: 2
Dashboard: http://127.0.0.1:38347/status,Memory: 8.00 GiB
Nanny: tcp://127.0.0.1:45863,


In [12]:
#Open and retrieve the tag data required for the analysis
tag = open_tag(tag_root, tag_name)
tag

DataTree('None', parent=None)
│   Dimensions:  ()
│   Data variables:
│       *empty*
│   Attributes:
│       pit_tag_id:       LT_A11338
│       scientific_name:  Dicentrarchus labrax
│       common_name:      European seabass
│       project:          BARGIP
├── DataTree('dst')
│       Dimensions:      (time: 676800)
│       Coordinates:
│         * time         (time) datetime64[ns] 5MB 2014-09-14T22:00:00 ... 2016-09-13...
│       Data variables:
│           temperature  (time) float64 5MB 25.56 25.56 25.55 ... 19.84 19.86 19.86
│           pressure     (time) float64 5MB 0.37 0.37 0.37 0.37 ... 12.25 12.12 12.31
└── DataTree('tagging_events')
        Dimensions:     (event_name: 2)
        Coordinates:
          * event_name  (event_name) object 16B 'release' 'fish_death'
        Data variables:
            time        (event_name) datetime64[ns] 16B 2014-09-15T14:08:00 2016-09-1...
            longitude   (event_name) float64 16B -2.585 -2.641
            latitude    (event_name) float64 16B 47.38 47.44

## 2. **Compare Reference Model with DST Tag Information:** Analyze and compare data from the reference model with information from the biologging data of the species in question. 

In this step, we compare the reference model data with Data Storage Tag information.
The process involves reading and cleaning the reference model, aligning time, converting depth units, subtracting tag data from the model, and saving the results.

In [13]:
# Import necessary libraries
import intake
from pangeo_fish.cf import bounds_to_bins
from pangeo_fish.diff import diff_z
from pangeo_fish.io import open_copernicus_catalog
from pangeo_fish.tags import adapt_model_time, reshape_by_bins, to_time_slice

# Drop data outside the reference interval
time_slice = to_time_slice(tag["tagging_events/time"])
time = tag["dst"].ds.time
cond = (time <= time_slice.stop) & (time >= time_slice.start)

tag_log = tag["dst"].ds.where(cond,drop=True)

min_ = tag_log.time[0]
max_ = tag_log.time[-1]

time_slice = slice(min_.data, max_.data)

In [14]:
def get_copernicus_zarr(product_id="IBI_MULTIYEAR_PHY_005_002"):
    master_cat = intake.open_catalog(catalog_url)
    if product_id == "IBI_MULTIYEAR_PHY_005_002":
        
        # Open necessary datasets
        sub_cat = master_cat[product_id]
        thetao = sub_cat["cmems_mod_ibi_phy_my_0.083deg-3D_P1D-m"](chunk="time").to_dask()[["thetao"]]
        zos = sub_cat["cmems_mod_ibi_phy_my_0.083deg-3D_P1D-m"](chunk="time").to_dask().zos
        deptho = sub_cat["cmems_mod_ibi_phy_my_0.083deg-3D_static"].to_dask().deptho
    
    # Assign latitude and longitude from thetao to deptho to shift in positions
    deptho["latitude"] = thetao["latitude"]
    deptho["longitude"] = thetao["longitude"]
    
    # Create mask for deptho
    mask = deptho.isnull()

    # Merge datasets and assign relevant variables
    ds = (
        thetao.rename({"thetao": "TEMP"})
        .assign(
            {
                "XE": zos,
                "H0": deptho,
                "mask": mask,
            }
        )
    ).rename({"latitude": "lat", "longitude": "lon", "elevation": "depth"})

    # Ensure depth is positive
    ds["depth"] = abs(ds["depth"])

    # Rearrange depth coordinates and assign dynamic depth and bathymetry
    ds = ds.isel(depth=slice(None, None, -1)).assign(
        {
            "dynamic_depth": lambda ds: (ds["depth"] + ds["XE"]).assign_attrs(
                {"units": "m", "positive": "down"}
            ),
            "dynamic_bathymetry": lambda ds: (ds["H0"] + ds["XE"]).assign_attrs(
                {"units": "m", "positive": "down"}
            ),
        }
    ).pipe(broadcast_variables, {"lat": "latitude", "lon": "longitude"})
    # print(uris_by_key)
    return ds

In [15]:
# Verify the data
import hvplot.xarray
import cmocean
from pangeo_fish.io import save_html_hvplot
plot=((-tag['dst'].pressure).hvplot(width=1000,height=500,color='blue')*(-tag_log).hvplot.scatter(x='time',y='pressure',color='red',size=5,width=1000,height=500)
      *((tag['dst'].temperature).hvplot(width=1000,height=500,color='blue')*(tag_log).hvplot.scatter(x='time',y='temperature',color='red',size=5,width=1000,height=500)))
filepath = f"{target_root}/tags.html"

save_html_hvplot(plot, filepath, storage_options)

# plot

(True, 'Plot saved successfully.')

In [16]:
from pangeo_fish.io import broadcast_variables

In [17]:
model = get_copernicus_zarr()

/srv/conda/envs/notebook/lib/python3.11/site-packages/intake_xarray/base.py:21: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  'dims': dict(self._ds.dims),
/srv/conda/envs/notebook/lib/python3.11/site-packages/intake_xarray/base.py:21: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  'dims': dict(self._ds.dims),


In [18]:
# Subset the reference_model by 
# - align model time with the time of tag_log, also
# - drop data for depth later that are unlikely due to the observed pressure from tag_log
# - defined latitude and longitude of bbox.  
#
reference_model = (
    model.sel(time=adapt_model_time(time_slice))
    .sel(lat=slice(*bbox["latitude"]), lon=slice(*bbox["longitude"]))
    .pipe(
        lambda ds: ds.sel(
            depth=slice(None, (tag_log["pressure"].max() - ds["XE"].min()).compute())
        )
    )
)

In [19]:
%%time
# Reshape the tag log, so that it bins to the time step of reference_model
reshaped_tag = reshape_by_bins(
    tag_log,
    dim="time",
    bins=(
        reference_model.cf.add_bounds(["time"], output_dim="bounds")
        .pipe(bounds_to_bins, bounds_dim="bounds")
        .get("time_bins")
    ),
    bin_dim="bincount",
    other_dim="obs",
).chunk({"time": chunk_time})

CPU times: user 2.78 s, sys: 171 ms, total: 2.95 s
Wall time: 2.87 s


In [20]:
# Subtract the time_bined tag_log from the reference_model. 
# Here, for each time_bin, each observed value are compared with the correspoindng depth of reference_model using diff_z function.  
#

diff = (
    diff_z(reference_model.chunk(dict(depth=-1)), reshaped_tag, depth_threshold=relative_depth_threshold)
    .assign_attrs({"tag_id": tag_name})
    .assign(
        {
            "H0": reference_model["H0"],
            "ocean_mask": reference_model["H0"].notnull(),
        }
    )
)

# Persist the diff data
diff = diff.chunk(default_chunk).persist()
# diff

/srv/conda/envs/notebook/lib/python3.11/site-packages/distributed/client.py:3157: UserWarning: Sending large graph of size 12.21 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


In [21]:
%%time
# Verify the data
# diff["diff"].count(["lat","lon"]).plot()

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.77 µs


In [22]:
target_lat = diff["lat"]
target_lon = diff["lon"]

In [23]:
%%time
# Save snapshot to disk
diff.to_zarr(
    f"{target_root}/diff.zarr", mode="w", storage_options=storage_options
)

# Cleanup
del tag_log, model, reference_model, reshaped_tag, diff

CPU times: user 10.1 s, sys: 1.31 s, total: 11.4 s
Wall time: 2min 47s


## 3. **Regrid the Grid from Reference Model Grid to Healpix Grid:** Transform the grid from the reference model to the Healpix grid for further analysis.

In this step, we regrid the data from the reference model grid to a Healpix grid. This process involves defining the Healpix grid, creating the target grid, computing interpolation weights, performing the regridding, and saving the regridded data.


In [ ]:
# Import necessary libraries
import s3fs
import numpy as np
from xarray_healpy import HealpyGridInfo, HealpyRegridder
from pangeo_fish.grid import center_longitude

In [ ]:
%%time

# Open the diff data and performs cleaning operations to prepare it for regridding.

ds = (
    xr.open_dataset(f"{target_root}/diff.zarr", engine="zarr", chunks={},
                    storage_options=storage_options)
    .pipe(lambda ds: ds.merge(ds[["latitude", "longitude"]].compute()))
    .swap_dims({"lat": "yi", "lon": "xi"})
)
ds

In [ ]:
s3 = s3fs.S3FileSystem(
    anon=False,
    client_kwargs={
        "endpoint_url": "https://s3.gra.perf.cloud.ovh.net",
    },
)

In [ ]:
coastal_distance = xr.open_zarr(distance_filepath).sel(lat = slice(56,40),lon=slice(-13,5))

In [ ]:
coastal_distance = coastal_distance.sortby("lat")

In [ ]:
coastal_distance = coastal_distance.interp(lat=target_lat, lon=target_lon, method='linear')

In [ ]:
coastal_distance["dist"] = (1+np.exp(-(coastal_distance.dist*coastal_distance.dist)*distance_scale_factor))

In [ ]:
coastal_distance = (
    coastal_distance
    .swap_dims({"lat": "yi", "lon": "xi"})
    .drop_vars(["lat", "lon"])
)

In [ ]:
%%time
# Define the target Healpix grid information
grid = HealpyGridInfo(level=int(np.log2(nside)), rot=rot)
target_grid = grid.target_grid(ds).pipe(center_longitude, 0)
target_grid

In [ ]:
%%time
# Compute the interpolation weights for regridding the diff data
regridder = HealpyRegridder(
    ds[["longitude", "latitude", "ocean_mask"]],
    target_grid,
    method="bilinear",
    interpolation_kwargs={"mask": "ocean_mask", "min_vertices": min_vertices},
)
regridder

In [ ]:
%%time
# Perform the regridding operation using the computed interpolation weights.
regridded = regridder.regrid_ds(ds)
regridded

In [ ]:
regridded_coastal = regridder.regrid_ds(coastal_distance)

In [ ]:
%%time
# Reshape the regridded data to 2D
reshaped = grid.to_2d(regridded).pipe(center_longitude, 0)
reshaped = reshaped.persist()
reshaped

In [ ]:
reshaped_coastal = grid.to_2d(regridded_coastal).pipe(center_longitude,0)

In [ ]:
# This cell verifies the regridded data by plotting the count of non-NaN values.
# reshaped["diff"].count(["x", "y"]).plot()

In [ ]:
coastal_chunk = {"x" : default_chunk_xy["x"], "y":default_chunk_xy["y"]}

In [ ]:
reshaped["diff"].isel(time=0).hvplot.quadmesh(title = "Carte des différences avant l'ajout de l'incertitude",x='longitude',y="latitude",cmap="cool",coastline="10m",xlim=bbox["longitude"],ylim=bbox['latitude'])

In [ ]:
reshaped["diff"] = reshaped["diff"] / reshaped_coastal["dist"]

In [ ]:
reshaped["diff"].isel(time=0).hvplot.quadmesh(title = "Carte des différences après l'ajout de l'incertitude",x='longitude',y="latitude",cmap="cool",coastline="10m",xlim=bbox["longitude"],ylim=bbox['latitude'])

In [49]:
%%time
# This cell saves the regridded data to Zarr format, then cleans up unnecessary variables to free up memory after the regridding process.  
reshaped.chunk(default_chunk_xy).to_zarr(
    f"{target_root}/diff-regridded.zarr",
    mode="w",
    consolidated=True,
    compute=True,
    storage_options=storage_options,
)

reshaped_coastal.chunk(coastal_chunk).to_zarr(
    f"{target_root}/coastal.zarr",
    mode="w",
    consolidated=True,
    compute=True,
    storage_options=storage_options,
)
# Cleanup unnecessary variables to free up memory
del ds, grid, target_grid, regridder, regridded, reshaped,reshaped_coastal

/srv/conda/envs/notebook/lib/python3.11/site-packages/distributed/client.py:3157: UserWarning: Sending large graph of size 41.57 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
/srv/conda/envs/notebook/lib/python3.11/site-packages/distributed/client.py:3157: UserWarning: Sending large graph of size 168.94 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


CPU times: user 340 ms, sys: 541 ms, total: 880 ms
Wall time: 2.19 s


## 4. **Construct Emission Matrix:** Create an emission matrix based on the transformed grid.

In this step, we construct the emission probability matrix based on the differences between the observed tag temperature and the reference sea temperature computed in Workflow 2 and regridded in Workflow 3. The emission probability matrix represents the likelihood of observing a specific temperature difference given the model parameters and configurations.


In [40]:
# Import necessary libraries
from toolz.dicttoolz import valfilter
from pangeo_fish.distributions import create_covariances, normal_at
from pangeo_fish.pdf import normal


In [41]:
%%time
# Open the regridded diff data 
differences = xr.open_dataset(
    f"{target_root}/diff-regridded.zarr",
    engine="zarr",
    chunks={},
    storage_options=storage_options,
).pipe(lambda ds: ds.merge(ds[["latitude", "longitude"]].compute()))
differences

CPU times: user 82.8 ms, sys: 67 ms, total: 150 ms
Wall time: 255 ms


<xarray.Dataset> Size: 1GB
Dimensions:     (y: 1213, x: 1497, time: 71)
Coordinates:
    cell_ids    (y, x) int64 15MB 56625549 56625551 ... 64507991 64507997
    latitude    (y, x) float64 15MB 30.67 30.68 30.69 30.7 ... 62.88 62.9 62.91
    longitude   (y, x) float64 15MB -4.003 -4.014 -4.025 ... -1.244 -1.257 -1.27
    resolution  float64 8B 0.0002498
  * time        (time) datetime64[ns] 568B 2014-06-07 2014-06-08 ... 2014-08-16
Dimensions without coordinates: y, x
Data variables:
    H0          (y, x) float64 15MB dask.array<chunksize=(1213, 1497), meta=np.ndarray>
    diff        (time, y, x) float64 1GB dask.array<chunksize=(24, 1213, 1497), meta=np.ndarray>
    ocean_mask  (y, x) float64 15MB dask.array<chunksize=(1213, 1497), meta=np.ndarray>
Attributes:
    grid_type:  healpix
    lat:        0
    level:      12
    lon:        30
    nside:      4096
    rot_lat:    0
    rot_lon:    30

In [42]:
%%time
# Compute initial and final position
grid = differences[["latitude", "longitude"]].compute()

initial_position = tag["tagging_events"].ds.sel(event_name="release")
cov = create_covariances(1e-6, coord_names=["latitude", "longitude"])
initial_probability = normal_at(
    grid, pos=initial_position, cov=cov, normalize=True, axes=["latitude", "longitude"]
)

final_position = tag["tagging_events"].ds.sel(event_name="fish_death")
if final_position[["longitude", "latitude"]].to_dataarray().isnull().all():
    final_probability = None
else:
    cov = create_covariances(recapture_std**2, coord_names=["latitude", "longitude"])
    final_probability = normal_at(
        grid,
        pos=final_position,
        cov=cov,
        normalize=True,
        axes=["latitude", "longitude"],
    )


CPU times: user 154 ms, sys: 18.5 ms, total: 172 ms
Wall time: 162 ms


In [43]:
%%time
#compute emission probability matrix

emission_pdf = (
    normal(differences["diff"], mean=0, std=differences_std, dims=["y", "x"])
    .to_dataset(name="pdf")
    .assign(
        valfilter(
            lambda x: x is not None,
            {
                "initial": initial_probability,
                "final": final_probability,
                "mask": differences["ocean_mask"],
            },
        )
    )
    .assign_attrs(differences.attrs )#| {"max_sigma": max_sigma})
)

emission_pdf=emission_pdf.chunk(default_chunk_xy).persist()
emission_pdf

CPU times: user 20.7 ms, sys: 1.48 ms, total: 22.2 ms
Wall time: 18.5 ms


/srv/conda/envs/notebook/lib/python3.11/site-packages/distributed/client.py:3157: UserWarning: Sending large graph of size 69.28 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


<xarray.Dataset> Size: 1GB
Dimensions:     (y: 1213, x: 1497, time: 71)
Coordinates:
    cell_ids    (y, x) int64 15MB dask.array<chunksize=(1213, 1497), meta=np.ndarray>
    latitude    (y, x) float64 15MB dask.array<chunksize=(1213, 1497), meta=np.ndarray>
    longitude   (y, x) float64 15MB dask.array<chunksize=(1213, 1497), meta=np.ndarray>
    resolution  float64 8B 0.0002498
  * time        (time) datetime64[ns] 568B 2014-06-07 2014-06-08 ... 2014-08-16
Dimensions without coordinates: y, x
Data variables:
    pdf         (time, y, x) float64 1GB dask.array<chunksize=(24, 1213, 1497), meta=np.ndarray>
    initial     (y, x) float64 15MB dask.array<chunksize=(1213, 1497), meta=np.ndarray>
    final       (y, x) float64 15MB dask.array<chunksize=(1213, 1497), meta=np.ndarray>
    mask        (y, x) float64 15MB dask.array<chunksize=(1213, 1497), meta=np.ndarray>
Attributes:
    grid_type:  healpix
    lat:        0
    level:      12
    lon:        30
    nside:      4096
    rot_lat:    0
    rot_lon:    30

In [44]:
#Verify the data
# emission_pdf["pdf"].count(["x", "y"]).plot()

In [45]:
# This cell saves the emission data to Zarr format, then cleans up unnecessary variables to free up memory.

emission_pdf.to_zarr(
    f"{target_root}/emission.zarr", mode="w", consolidated=True,
    storage_options=storage_options,
)

del differences, grid, initial_probability, final_probability, emission_pdf

## 5. **Replace emission for the tags with warm spikes detected**

In [46]:
%%time 
# Import necessary libraries and open data and perform initial setup
from pangeo_fish import acoustic, utils
import hvplot.xarray
import pandas as pd
from pangeo_fish.heat import heat_regulation, powerpalnt_emission_map
emission = xr.open_dataset(
    f"{target_root}/emission.zarr", engine="zarr", chunks={},#"x": -1, "y": -1},
    storage_options=storage_options,
)
emission

CPU times: user 62.5 ms, sys: 6.58 ms, total: 69.1 ms
Wall time: 66.4 ms


<xarray.Dataset> Size: 1GB
Dimensions:     (y: 1213, x: 1497, time: 71)
Coordinates:
    cell_ids    (y, x) int64 15MB dask.array<chunksize=(1213, 1497), meta=np.ndarray>
    latitude    (y, x) float64 15MB dask.array<chunksize=(1213, 1497), meta=np.ndarray>
    longitude   (y, x) float64 15MB dask.array<chunksize=(1213, 1497), meta=np.ndarray>
    resolution  float64 8B ...
  * time        (time) datetime64[ns] 568B 2014-06-07 2014-06-08 ... 2014-08-16
Dimensions without coordinates: y, x
Data variables:
    final       (y, x) float64 15MB dask.array<chunksize=(1213, 1497), meta=np.ndarray>
    initial     (y, x) float64 15MB dask.array<chunksize=(1213, 1497), meta=np.ndarray>
    mask        (y, x) float64 15MB dask.array<chunksize=(1213, 1497), meta=np.ndarray>
    pdf         (time, y, x) float64 1GB dask.array<chunksize=(24, 1213, 1497), meta=np.ndarray>
Attributes:
    grid_type:  healpix
    lat:        0
    level:      12
    lon:        30
    nside:      4096
    rot_lat:    0
    rot_lon:    30

In [47]:
powerplant_flag=False

In [48]:
if powerplant_flag :
    # Loading detections, formatting and reducing observation window
    detections = pd.read_csv(detection_file).set_index("time").to_xarray()
    detections["time"]=detections["time"].astype("datetime64")
    detections = detections.sel(time=emission["time"]) # Narrowing the data to the observed days only
    
    pp_map = pd.read_csv(powerplant_file,sep=";").drop("Country",axis=1).to_xarray() # Loading powerplant locations data
    
    # Combining and replacing the emission map at the given timestamps for the days where warm plume are detected
    combined_masks = powerpalnt_emission_map(pp_map,emission,buffer_size,rot)
    emission = heat_regulation(emission,detections,combined_masks)

## 6. **Combine and Normalize Emission Matrix:** Merge the emission matrix and normalize it for further processing.

In this step, we combine the emission probability matrix constructed in Workflow 4 and 5 then normalize it to ensure that the probabilities sum up to one. This step prepares the combined emission matrix for further analysis and interpretation.


In [49]:
# Import necessary libraries 
from pangeo_fish.pdf import combine_emission_pdf
import hvplot.xarray

In [50]:
# Open and combine the emission probability matrix

combined = (
    emission
    .pipe(combine_emission_pdf)
    .chunk(default_chunk_xy)
    .persist()  # convert to comment if the emission matrix does *not* fit in memory
)
combined

<xarray.Dataset> Size: 1GB
Dimensions:     (y: 1213, x: 1497, time: 71)
Coordinates:
    cell_ids    (y, x) int64 15MB dask.array<chunksize=(1213, 1497), meta=np.ndarray>
    latitude    (y, x) float64 15MB dask.array<chunksize=(1213, 1497), meta=np.ndarray>
    longitude   (y, x) float64 15MB dask.array<chunksize=(1213, 1497), meta=np.ndarray>
    resolution  float64 8B 0.0002498
  * time        (time) datetime64[ns] 568B 2014-06-07 2014-06-08 ... 2014-08-16
Dimensions without coordinates: y, x
Data variables:
    initial     (y, x) float64 15MB dask.array<chunksize=(1213, 1497), meta=np.ndarray>
    final       (y, x) float64 15MB dask.array<chunksize=(1213, 1497), meta=np.ndarray>
    mask        (y, x) float64 15MB dask.array<chunksize=(1213, 1497), meta=np.ndarray>
    pdf         (time, y, x) float64 1GB dask.array<chunksize=(24, 1213, 1497), meta=np.ndarray>
Attributes:
    grid_type:  healpix
    lat:        0
    level:      12
    lon:        30
    nside:      4096
    rot_lat:    0
    rot_lon:    30

In [51]:
# Verify the data and visualize the sum of probabilities
# combined["pdf"].sum(["x", "y"]).hvplot(width=400)

In [52]:
# Save the combined and normalized emission matrix
combined.to_zarr(
    f"{target_root}/combined.zarr", mode="w", consolidated=True,
    storage_options=storage_options    
)
del combined

## 7. **Estimate Model Parameters:** Determine the parameters of the model based on the normalized emission matrix.

This step first estimates maxixmum allowed value of  model parameter 'sigma' max_sigma.  Then we
create an optimizer with an expected parameter range, fitting the model to the normalized emission matrix.  
The resulting optimized parameters is saved to a json file.  

In [53]:
# Import necessary libraries and modules for data analysis.
import xarray as xr
import pandas as pd
from pangeo_fish.hmm.estimator import EagerScoreEstimator
from pangeo_fish.hmm.optimize import EagerBoundsSearch
from pangeo_fish.utils import temporal_resolution
# Open the data
emission = (
    xr.open_dataset(
        f"{target_root}/combined.zarr",
        engine="zarr",
        chunks={},
        inline_array=True,
        storage_options=storage_options            
    )
)
emission

<xarray.Dataset> Size: 1GB
Dimensions:     (y: 1213, x: 1497, time: 71)
Coordinates:
    cell_ids    (y, x) int64 15MB dask.array<chunksize=(1213, 1497), meta=np.ndarray>
    latitude    (y, x) float64 15MB dask.array<chunksize=(1213, 1497), meta=np.ndarray>
    longitude   (y, x) float64 15MB dask.array<chunksize=(1213, 1497), meta=np.ndarray>
    resolution  float64 8B ...
  * time        (time) datetime64[ns] 568B 2014-06-07 2014-06-08 ... 2014-08-16
Dimensions without coordinates: y, x
Data variables:
    final       (y, x) float64 15MB dask.array<chunksize=(1213, 1497), meta=np.ndarray>
    initial     (y, x) float64 15MB dask.array<chunksize=(1213, 1497), meta=np.ndarray>
    mask        (y, x) float64 15MB dask.array<chunksize=(1213, 1497), meta=np.ndarray>
    pdf         (time, y, x) float64 1GB dask.array<chunksize=(24, 1213, 1497), meta=np.ndarray>
Attributes:
    grid_type:  healpix
    lat:        0
    level:      12
    lon:        30
    nside:      4096
    rot_lat:    0
    rot_lon:    30

In [54]:
# Compute maximum displacement for each reference model time step
# and estimate maximum sigma value for limiting the optimisation step

earth_radius_ = xr.DataArray(earth_radius, dims=None)

timedelta = temporal_resolution(emission["time"]).pint.quantify().pint.to("h")
grid_resolution = earth_radius_ * emission["resolution"].pint.quantify()

maximum_speed_ = xr.DataArray(maximum_speed, dims=None).pint.to("km / h")
max_grid_displacement = maximum_speed_ * timedelta * adjustment_factor / grid_resolution
max_sigma = max_grid_displacement.pint.to("dimensionless").pint.magnitude / truncate
emission.attrs["max_sigma"]=max_sigma
max_sigma

/srv/conda/envs/notebook/lib/python3.11/site-packages/pangeo_fish/utils.py:122: FutureWarning: Day.delta is deprecated and will be removed in a future version. Use pd.Timedelta(obj) instead
  timedelta = to_offset(freq).delta.to_numpy()


47.11935070054214

In [55]:
# Create and configure estimator and optimizer
emission = emission.compute()  # Convert to comment if the emission matrix does *not* fit in memory
estimator = EagerScoreEstimator()
optimizer = EagerBoundsSearch(
    estimator,
    (1e-4, emission.attrs["max_sigma"]),
    optimizer_kwargs={"disp": 3, "xtol": tolerance},
)

In [56]:
%%time
# Fit the model parameter to the data
optimized = optimizer.fit(emission)

 
 Func-count     x          f(x)          Procedure
    1        17.9981      1070.72        initial
    2        29.1214      1034.67        golden
    3         35.996      1011.86        golden
    4        40.2447      1008.29        golden
    5        40.0071      1008.43        parabolic
    6        42.8706      1006.95        golden
    7        44.4935      1005.99        golden
    8        45.4965      1005.48        golden
    9        46.1164      1005.26        golden
   10        46.4995      1005.13        golden
   11        46.7362      1005.06        golden
   12        46.8826      1005.02        golden
   13         46.973         1005        golden
   14        47.0289      1004.98        golden
   15        47.0635      1004.97        golden
   16        47.0848      1004.97        golden
   17         47.098      1004.97        golden
   18        47.1062      1004.96        golden
   19        47.1112      1004.96        golden
   20        47.1143      1004.

In [57]:
# Save the optimized parameters
params=optimized.to_dict()
pd.DataFrame.from_dict(params, orient='index').to_json(f"{target_root}/parameters.json", 
                                                               storage_options=storage_options    )       

In [58]:
# Cleanup
del optimized,  emission 

## 8. **Compute State Probabilities and Tracks:** Calculate the probability distribution of the species in question and compute the tracks.

This step involves predicting state probabilities using the optimised parameter sigma computed in the last step together with normalized emission matrix.  

In [59]:
# Import necessary libraries and modules for data analysis.
import xarray as xr
import pandas as pd
import hvplot.xarray
from pangeo_fish.hmm.estimator import EagerScoreEstimator
from pangeo_fish.io import save_trajectories

# Recreate the Estimator
params=pd.read_json(f"{target_root}/parameters.json", storage_options=storage_options    ).to_dict()[0]
optimized = EagerScoreEstimator(**params)
optimized

EagerScoreEstimator(sigma=47.1189494021, truncate=4.0)

In [60]:
%%time
# Load the Data
emission = (
    xr.open_dataset(
        f"{target_root}/combined.zarr",
        engine="zarr",
        chunks=default_chunk_xy,
        inline_array=True,
        storage_options=storage_options            
    ).compute()
)

# Predict the State Probabilities

states = optimized.predict_proba(emission)
states = states.to_dataset().chunk(default_chunk_xy).persist()
states

CPU times: user 1min 16s, sys: 2.63 s, total: 1min 19s
Wall time: 1min 13s


/srv/conda/envs/notebook/lib/python3.11/site-packages/distributed/client.py:3157: UserWarning: Sending large graph of size 1.00 GiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


<xarray.Dataset> Size: 1GB
Dimensions:     (y: 1213, x: 1497, time: 71)
Coordinates:
    cell_ids    (y, x) int64 15MB dask.array<chunksize=(1213, 1497), meta=np.ndarray>
    latitude    (y, x) float64 15MB dask.array<chunksize=(1213, 1497), meta=np.ndarray>
    longitude   (y, x) float64 15MB dask.array<chunksize=(1213, 1497), meta=np.ndarray>
    resolution  float64 8B 0.0002498
  * time        (time) datetime64[ns] 568B 2014-06-07 2014-06-08 ... 2014-08-16
Dimensions without coordinates: y, x
Data variables:
    states      (time, y, x) float64 1GB dask.array<chunksize=(24, 1213, 1497), meta=np.ndarray>

In [61]:
# Verify the data and visualize the sum of probabilities
# states.sum(["x", "y"]).hvplot() +states.count(["x", "y"]).hvplot()

In [62]:
%%time
# Save probability distirbution, state matrix.  
states.chunk(default_chunk_xy).to_zarr(
    f"{target_root}/states.zarr", mode="w", consolidated=True,  
        storage_options=storage_options                
)

CPU times: user 292 ms, sys: 1.28 s, total: 1.57 s
Wall time: 1.83 s


In [63]:
%%time 
#decode tracks

trajectories = optimized.decode(
    emission,
    states.fillna(0),
    mode=track_modes,
    progress=False,
    additional_quantities=additional_track_quantities,
)
trajectories

/srv/conda/envs/notebook/lib/python3.11/site-packages/xarray/core/variable.py:2368: PerformanceWarning: Reshaping is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array.reshape(shape)

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array.reshape(shape)Explicitly passing ``limit`` to ``reshape`` will also silence this warning
    >>> array.reshape(shape, limit='128 MiB')
  stacked = self.stack({newdimname: dim})


CPU times: user 302 ms, sys: 52.2 ms, total: 354 ms
Wall time: 1 s


TrajectoryCollection with 2 trajectories

In [64]:
# Save trajectories. 
# Here we can chose format parquet for loading files from 'R'
# or chose to  format 'geoparquet' for further analysis of tracks using 
# geopands. 

save_trajectories(trajectories, target_root,storage_options, format="parquet")

In [65]:
# Cleanup
del optimized,  emission, states, trajectories

In [66]:
cluster.close()
client.close()

## 9. **Visualization:** Visualize the results of the analysis for interpretation and insight.


In this step, we visualize various aspects of the analysis results to gain insights and interpret the model outcomes. We plot the emission matrix, which represents the likelihood of observing a specific temperature difference given the model parameters and configurations. Additionally, we visualize the state probabilities, showing the likelihood of the system being in different states at each time step. We also plot each of the tracks of the tagged fish, displaying their movement patterns over time. Finally, we create a movie that combines the emission matrix and state probabilities to provide a comprehensive visualization of the analysis results.
